In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
import os
from keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
img_size = 128

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
def getLabel(input_string):
    try:
        i=input_string.index('-101')
        return input_string[4:i]
    except ValueError:
        return input_string[4:]

In [4]:
path = 'C:\\Users\\pc\\Desktop\\GP\\256_ObjectCategories\\256_ObjectCategories\\'
os.chdir(path)
folders = os.listdir()
folder_paths = []
all_images = []
all_classes = []

for folder in range(len(folders)):
    folder_paths = path+str(folders[folder])+str('\\')
    os.chdir(folder_paths)
    image_in_folder = os.listdir()
    label = getLabel(folders[folder])
    for i in range(len(image_in_folder)):
        img_path = folder_paths+str(image_in_folder[i])
        img = image.load_img(img_path, target_size = (64, 64))
        img_tensor = image.img_to_array(img)
        img_tensor = np.expand_dims(img_tensor, axis = 0)
        all_images.extend(img_tensor/255)
        all_classes.append(label)

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()

All_images = np.array(all_images)
All_classes = np.array(all_classes)
X_train, X_test, y_train, y_test = train_test_split(All_images, All_classes, test_size=0.2)
X_Train = np.array(X_train)
X_Test = np.array(X_test)
Y_Train = np.array(y_train)
Y_Test = np.array(y_test)
Y_Train = onehotencoder.fit_transform(Y_Train.reshape(-1, 1)).toarray()
Y_Test = onehotencoder.fit_transform(Y_Test.reshape(-1, 1)).toarray()
print(X_Train.shape)
print(Y_Train.shape)
print(X_Test.shape)
print(Y_Test.shape)

(24485, 64, 64, 3)
(24485, 257)
(6122, 64, 64, 3)
(6122, 257)


In [6]:
input_shape = (64, 64, 3)
num_classes = 257

model = Sequential([
    Conv2D(16, (5, 5), strides=(2, 2), padding='same', kernel_initializer='he_uniform', use_bias=False, 
        input_shape=input_shape), 
    BatchNormalization(scale=False),
    Activation('relu'),
    
    Conv2D(32, (3, 3), padding='same', kernel_initializer='he_uniform', use_bias=False),   
    BatchNormalization(scale=False),
    Activation('relu'),
    
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'),  
    Dropout(0.2),

    Conv2D(32, (1, 1), padding='same', kernel_initializer='he_uniform', use_bias=False), 
    BatchNormalization(scale=False),
    Activation('relu'),
    
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'), 
    Dropout(0.2),
    
    Flatten(), 
    Dense(512, kernel_initializer='he_uniform'), 
    Activation('relu'), 
    Dropout(0.2),
    Dense(num_classes, kernel_initializer='he_uniform'), 
    Activation('softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

model.count_params()

1188001

In [7]:
model.fit(X_Train, Y_Train, batch_size = 128, epochs = 50)

Epoch 1/50
192/192 [==============================] - 56s 220ms/step - loss: 5.5326 - accuracy: 0.0538
Epoch 2/50
192/192 [==============================] - 42s 217ms/step - loss: 4.8146 - accuracy: 0.1080
Epoch 3/50
192/192 [==============================] - 42s 218ms/step - loss: 4.4982 - accuracy: 0.1376
Epoch 4/50
192/192 [==============================] - 41s 215ms/step - loss: 4.2987 - accuracy: 0.1600
Epoch 5/50
192/192 [==============================] - 42s 220ms/step - loss: 4.1479 - accuracy: 0.1776
Epoch 6/50
192/192 [==============================] - 42s 216ms/step - loss: 4.0498 - accuracy: 0.1859
Epoch 7/50
192/192 [==============================] - 44s 227ms/step - loss: 3.9411 - accuracy: 0.1987
Epoch 8/50
192/192 [==============================] - 44s 228ms/step - loss: 3.8625 - accuracy: 0.2046
Epoch 9/50
192/192 [==============================] - 42s 221ms/step - loss: 3.7981 - accuracy: 0.2162
Epoch 10/50
192/192 [==============================] - 44s 227ms/step - l

In [8]:
test_loss, test_acc = model.evaluate(X_Test, Y_Test)
print(test_loss)
print(test_acc)

192/192 [==============================] - 3s 10ms/step - loss: 3.7115 - accuracy: 0.2814
3.711475133895874
0.2814439833164215
